# core

> TODO: Migrate most logic from nbz/cli.py to here, and add tests. See https://github.com/pydanny/nbz/issues/5

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()